In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# 定义要处理的入射角列表
incident_angles = [10, 15]

# 薄膜折射率（默认值，需根据实际薄膜类型修改）
n2 = 1.5
c = 3e10  # 光速（cm/s，若需波长单位转换可使用）

# 读取附件1数据
file_path = "附件1.xlsx"  # 请替换为附件1实际路径
try:
    df = pd.read_excel(file_path, sheet_name="Sheet1")
except FileNotFoundError:
    # 假设如果没有附件1，创建一个示例DataFrame
    data = {
        "波数 (cm-1)": [400, 800, 1200],
        "反射率 (%)": [50, 60, 70]
    }
    df = pd.DataFrame(data)

# 数据预处理：波数(q)、反射率（转换为小数）
df["波数_q(cm-1)"] = df["波数 (cm-1)"]
df["反射率_R(小数)"] = df["反射率 (%)"] / 100  # 转换为0-1范围
df["波长_λ(cm)"] = 1 / df["波数_q(cm-1)"]  # 波长=1/波数（cm）

# ----------------------核心公式定义----------------------
def calculate_n1_vertical(R12, n2):
    if R12 < 0 or R12 >= 1:
        return np.nan
    sqrt_R = np.sqrt(R12)
    n1 = n2 * (1 + sqrt_R) / (1 - sqrt_R)
    return n1

def calculate_n1_s_polarized(Rs12, n2, i_rad):
    if Rs12 < 0 or Rs12 >= 1:
        return np.nan
    sqrt_Rs = np.sqrt(Rs12)
    cosi = np.cos(i_rad)
    n1 = n2 * (sqrt_Rs + 1) / (1 - sqrt_Rs)  
    for _ in range(5):
        sinr = (n1 * np.sin(i_rad)) / n2
        if sinr > 1:
            return np.nan
        cosr = np.sqrt(1 - sinr**2)
        n1 = n2 * (sqrt_Rs * cosr + cosi) / (cosi - sqrt_Rs * cosr)
    return n1

def calculate_n1_p_polarized(Rp12, n2, i_rad):
    if Rp12 < 0 or Rp12 >= 1:
        return np.nan
    sqrt_Rp = np.sqrt(Rp12)
    cosi = np.cos(i_rad)
    n1 = n2 * (1 + sqrt_Rp) / (1 - sqrt_Rp)  
    for _ in range(5):
        sinr = (n1 * np.sin(i_rad)) / n2
        if sinr > 1:
            return np.nan
        cosr = np.sqrt(1 - sinr**2)
        n1 = n2 * (cosi + sqrt_Rp * cosr) / (sqrt_Rp * cosi + cosr)
    return n1

def calculate_thickness_e(n1, n2, λ, i_rad):
    sinr = (n1 * np.sin(i_rad)) / n2
    if sinr > 1:
        return np.nan
    cosr = np.sqrt(1 - sinr**2)
    m = 1
    e = (m * λ) / (2 * n2 * cosr)
    return e * 1e4  # 转换为μm（1cm = 1e4 μm）

In [2]:
# 处理每个入射角
for angle in incident_angles:
    i_rad = np.radians(angle)
    temp_df = df.copy()
    
    # 计算各场景结果
    # 垂直入射
    temp_df["垂直入射_n1"] = temp_df["反射率_R(小数)"].apply(lambda x: calculate_n1_vertical(x, n2))
    temp_df["垂直入射_e(μm)"] = temp_df.apply(
        lambda row: calculate_thickness_e(row["垂直入射_n1"], n2, row["波长_λ(cm)"], 0),
        axis=1
    )
    
    # 斜入射-s偏振
    temp_df["斜入射s偏振_n1"] = temp_df["反射率_R(小数)"].apply(lambda x: calculate_n1_s_polarized(x, n2, i_rad))
    temp_df["斜入射s偏振_e(μm)"] = temp_df.apply(
        lambda row: calculate_thickness_e(row["斜入射s偏振_n1"], n2, row["波长_λ(cm)"], i_rad),
        axis=1
    )
    
    # 斜入射-p偏振
    temp_df["斜入射p偏振_n1"] = temp_df["反射率_R(小数)"].apply(lambda x: calculate_n1_p_polarized(x, n2, i_rad))
    temp_df["斜入射p偏振_e(μm)"] = temp_df.apply(
        lambda row: calculate_thickness_e(row["斜入射p偏振_n1"], n2, row["波长_λ(cm)"], i_rad),
        axis=1
    )
    

In [5]:
# 整理输出数据
sheet1_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "垂直入射_n1"]].copy()
sheet1_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率R12(小数)", "空气折射率n1"]

sheet2_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "垂直入射_e(μm)"]].copy()
sheet2_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率R12(小数)", "薄膜厚度e(μm)"]

sheet3_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射s偏振_n1"]].copy()
sheet3_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rs12(小数)", "空气折射率n1"]

sheet4_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射s偏振_e(μm)"]].copy()
sheet4_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rs12(小数)", "薄膜厚度e(μm)"]

sheet5_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射p偏振_n1"]].copy()
sheet5_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rp12(小数)", "空气折射率n1"]

sheet6_data = temp_df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射p偏振_e(μm)"]].copy()
sheet6_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rp12(小数)", "薄膜厚度e(μm)"]
    
# 导出到Excel
wb = Workbook()
wb.remove(wb.active)
    
sheets = [
        ("垂直入射_空气折射率n1", sheet1_data),
        ("垂直入射_薄膜厚度e(μm)", sheet2_data),
        ("斜入射s偏振_空气折射率n1", sheet3_data),
        ("斜入射s偏振_薄膜厚度e(μm)", sheet4_data),
        ("斜入射p偏振_空气折射率n1", sheet5_data),
        ("斜入射p偏振_薄膜厚度e(μm)", sheet6_data)
    ]
    
for sheet_name, data in sheets:
        ws = wb.create_sheet(title=sheet_name)
        for r in dataframe_to_rows(data, index=False, header=True):
            ws.append(r)
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 20)
            ws.column_dimensions[column_letter].width = adjusted_width
    
output_path = f"折射率与薄膜厚度计算结果_入射角{angle}度(μm).xlsx"
wb.save(output_path)
print(f"入射角{angle}°的计算结果已保存至：{output_path}")

入射角15°的计算结果已保存至：折射率与薄膜厚度计算结果_入射角15度(μm).xlsx


计算完成！结果已保存至：折射率与薄膜厚度计算结果.xlsx
工作表说明：
1. 垂直入射_空气折射率n1：垂直入射下空气n1计算结果
2. 垂直入射_薄膜厚度e：垂直入射下薄膜厚度e计算结果（单位：μm）
3. 斜入射s偏振_空气折射率n1：斜入射s偏振下空气n1计算结果
4. 斜入射s偏振_薄膜厚度e：斜入射s偏振下薄膜厚度e计算结果（单位：μm）
5. 斜入射p偏振_空气折射率n1：斜入射p偏振下空气n1计算结果
6. 斜入射p偏振_薄膜厚度e：斜入射p偏振下薄膜厚度e计算结果（单位：μm）
